In [ ]:
# Necessary libraries

%matplotlib inline

import networkx as nx
import numpy as np

## Auxiliary functions

In [ ]:
def get_neighbors(graph, node, level):
    """Get neighbors of a given node up to a certain level"""
    # All neighbors up to the given level
    all_neighbors = nx.single_source_shortest_path_length(graph, node, cutoff=level)
    
    # Select only neighbors at the level we want
    return [k for (k, v) in all_neighbors.items() if v == level]

## Algorithm

In [ ]:
def generate_initial_conditions(number_of_nodes, randomness, quality):
    """
    Initial conditions for the simulation
    
    Set additional attributes for each node
    """
    # Network creation
    G = nx.generators.watts_strogatz_graph(number_of_nodes, 4, randomness)
    
    for n in G.nodes():
        node = G.node[n]
        node['adopter'] = 0             # 1 is adopter, 0 non-adopter
        node['preference'] = 0          # pi
        node['local_influence'] = 0     # xi
        
        # Set individual preference (yi)
        preference = np.random.random()
        if quality >= preference: 
            node['individual_preference'] = 1
        else:
            node['individual_preference'] = 0
        
        # Consumers at level 2
        node['neighbors_level_2'] = get_neighbors(G, n, level=2)
        
        # Total number of neighbors up to level 2
        node['number_of_neighbors'] = len(G.neighbors(n) + node['neighbors_level_2'])
    
    return G

In [ ]:
def evolution(graph, max_time, marketing_effort, social_threshold, social_influence, minimal_utility):
    def is_adopter(node):
        """Return True if a node is an adopter"""
        return graph.node[node]['adopter'] == 1
    
    for t in range(max_time):
        for n in graph.nodes():
            node = graph.node[n]

            # -- Adoption due to marketing
            p = np.random.random()
            if not node['adopter'] and (p < marketing_effort):
                node['adopter'] = 1

            # -- Adoption due to utility

            # Adopters at level 1
            adopters_level_1 = filter(is_adopter, graph.neighbors(n))

            # Adopters at level 2
            adopters_level_2 = filter(is_adopter, node['neighbors_level_2'])

            # Total number of adopters
            adopters_among_neighbors = len(adopters_level_1) + len(adopters_level_2)

            # Only if a consumer has adopters among his neighbors, he decides to adopt!
            if adopters_among_neighbors > 0:

                # Ai value
                adopters_percentaje = adopters_among_neighbors / node['number_of_neighbors']

                # Computing xi
                if adopters_percentaje > social_threshold:
                    node['local-influence'] = 1
                else:
                    node['local-influence'] = 0

                # Computing utility Ui
                utility = social_influence * node['local_influence'] + (1 - social_influence) * node['individual_preference']

                # print(utility)
                if utility > minimal_utility:
                    node['adopter'] = 1

In [ ]:
G = generate_initial_conditions(number_of_nodes=50,
                                randomness=0.01,
                                quality=0.5)

In [ ]:
pos = pos=nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos)
nx.draw_networkx_edges(G, pos)
nx.draw_networkx_labels(G, pos);

In [ ]:
G.node[6]